In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb
import requests
from bs4 import BeautifulSoup
import time

In [5]:
current_df = pd.read_csv("top_music.csv")

In [6]:
current_df.head()

,Year,Rank,Artist,Song,valence,danceability,energy,tempo
0,1960,1,percy faith,theme from a summer place,0.9130,0.325,0.4120,186.720
1,1960,2,jim reeves,hell have to go,0.1905,0.533,0.2435,81.068
2,1960,3,everly brothers,cathys clown,0.8660,0.498,0.5820,119.809
3,1960,4,johnny preston,running bear,NaN,NaN,NaN,NaN
4,1960,5,mark dinning,teen angel,0.3300,0.447,0.1820,95.094


In [14]:
years = list(range(1960,2017))
print(years)

[1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]


In [10]:
# recessions: https://en.wikipedia.org/wiki/List_of_recessions_in_the_United_States
# post-1960 recessions: 1960 - 61, 69-70, 73-75, 80, 81-82, 90-91, 07-09, 20
recession_years = set(
    list(range(1960, 1962)) +
    list(range(1969, 1971)) +
    list(range(1973, 1976)) +
    [1980] +
    list(range(1981, 1983)) +
    list(range(1990, 1992)) +
    list(range(2007, 2010))
)

print(recession_years)

{1990, 1991, 1960, 1961, 1969, 1970, 2007, 1973, 1974, 1975, 2008, 2009, 1980, 1981, 1982}


In [ ]:
# elections: 1960 >= 1960 + 4x > 2016
election_years = set()

for i in range(1956, 2020, 4):
    election_years.add(i)

print(election_years)

{1984, 2016, 1956, 1988, 1960, 1992, 2012, 1964, 1996, 1968, 2000, 1972, 2004, 2008, 1976, 1980}


In [17]:
dummy_data = {
    "year": years,
    "election": [1 if year in election_years else 0 for year in years],
    "recession": [1 if year in recession_years else 0 for year in years]
}

dummy_df = pd.DataFrame(dummy_data)
print(dummy_df)

    year  election  recession
0   1960         1          1
1   1961         0          1
2   1962         0          0
3   1963         0          0
4   1964         1          0
5   1965         0          0
6   1966         0          0
7   1967         0          0
8   1968         1          0
9   1969         0          1
10  1970         0          1
11  1971         0          0
12  1972         1          0
13  1973         0          1
14  1974         0          1
15  1975         0          1
16  1976         1          0
17  1977         0          0
18  1978         0          0
19  1979         0          0
20  1980         1          1
21  1981         0          1
22  1982         0          1
23  1983         0          0
24  1984         1          0
25  1985         0          0
26  1986         0          0
27  1987         0          0
28  1988         1          0
29  1989         0          0
30  1990         0          1
31  1991         0          1
32  1992  

In [ ]:
p4_merged_data = duckdb.sql(
    """SELECT * FROM current_df
    LEFT JOIN dummy_df
    ON current_df.Year = dummy_df.year""").df()

p4_merged_data.head()

,Year,Rank,Artist,Song,valence,danceability,energy,tempo,year_1,election,recession
0,1960,1,percy faith,theme from a summer place,0.9130,0.325,0.4120,186.720,1960,1,1
1,1960,2,jim reeves,hell have to go,0.1905,0.533,0.2435,81.068,1960,1,1
2,1960,3,everly brothers,cathys clown,0.8660,0.498,0.5820,119.809,1960,1,1
3,1960,4,johnny preston,running bear,NaN,NaN,NaN,NaN,1960,1,1
4,1960,5,mark dinning,teen angel,0.3300,0.447,0.1820,95.094,1960,1,1


In [25]:
p4_merged_data.to_csv("p4_dataset_1.csv")